In [1]:
%load_ext autoreload
%autoreload 2

# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd
import geopandas as gpd
import numpy as np
import pickle
pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames

# Ignorar los warnings
# -----------------------------------------------------------------------
import warnings
warnings.filterwarnings('ignore')

# Librería para el acceso a variables y funciones
# -----------------------------------------------------------------------
import sys
sys.path.append("../")
from src import soporte_rentabilidad as sr

In [117]:
data = gpd.read_file("../data/transformed/final_sale.geojson")

with open("../transformers/scaler.pkl", "rb") as f:
    scaler = pickle.load(f)

with open("../transformers/target_encoder.pkl", "rb") as f:
    encoder = pickle.load(f)

with open("../transformers/model.pkl", "rb") as f:
    model = pickle.load(f)

features = data[['tipo', 'exterior', 'planta', 'ascensor', 'tamanio',
                 'habitaciones', 'banios', 'aire_acondicionado', 'trastero',
                 'terraza', 'patio', 'distrito']]

df_encoded = encoder.transform(features)

dicc_sust_planta = {
    'st': -3, 
    'ss': -2,
    'bj': -1,
    'en': 0.5, 
    'ND': 0
}

df_encoded['planta'] = features['planta'].map(dicc_sust_planta).fillna(features['planta'])

lista_col_bools = ["ascensor", "exterior", "aire_acondicionado", "trastero", "terraza", "patio"]

for columna in lista_col_bools:
    df_encoded[columna] = features[columna].fillna('ND').astype(str).replace({"True": 1, "False": -1, "ND": 0})

df_encoded['tamanio'] = scaler.transform(features[['tamanio']])

data["alquiler_predicho"] = model.predict(df_encoded)

Skipping field urls_imagenes: unsupported OGR type: 5


In [118]:
data.head(1)

,codigo,precio,precio_por_zona,tipo,exterior,planta,ascensor,tamanio,habitaciones,banios,aire_acondicionado,trastero,terraza,patio,estado,direccion,descripcion,anunciante,contacto,distrito,geometry,alquiler_predicho
0,104792745,149900.0,1180.0,piso,True,4,True,127.0,3,2,False,False,False,True,good,"carretera de Huesca, 21","Junto a la Academia General Militar, en carret...",FINCAS RUIZ,876210884,Distrito Rural,POINT (-0.86935 41.6973),1180.285


In [119]:
data.to_file("prueba.geojson", driver="GeoJSON")